## Data Cleaning

 * Data Quality (validity, accuracy, completeness, consistency, uniformity)
 * The workflow 
     * Inspection (data profiling, visualizations, software packages)
     * Cleaning (irrelevant data, duplicates, type conver., syntax errors, 6 more)
     * Verifying
     * Reporting

[The Ultimate Guide to Data Cleaning](https://towardsdatascience.com/the-ultimate-guide-to-data-cleaning-3969843991d4) from Toward Data Science.

## Data Formats

* Data comes from various systems or databases
* Often dumped in some easy to generate format: CSV, JSON, XML
* Sometimes reading data not intended for re-use: text, audio/video metadata
* A few file formats specific to structured data interchange: HDF5

[How to read most commonly used data formats...](https://www.analyticsvidhya.com/blog/2017/03/read-commonly-used-formats-using-python/)

## Reading XML Data

- XML is a widely used file format for data on the web.
- It is actually a document markup language like HTML, so it can represent quite complex structures. 
- However it is often used to store simple tabular data as well.

Here's a [sample XML file](files/sample.xml).

We can use the Python [xml ElementTree module](https://docs.python.org/3.6/library/xml.etree.elementtree.html)
to parse XML data into a Python data structure.  In the case of this 
sample file, it consists of many &lt;record> elements each with four data points. We want to parse into 
a dataframe with four columns. 


In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
tree = ET.parse('files/sample.xml')
root = tree.getroot()
# print every row with xml tag and xml text 
for row in root:
    for child in row:
        print(child.tag, child.text)


Name Hedy
Date 2018-04-14T08:51:54-07:00
City Sijsele
Amount 105.06
Name Indigo
Date 2018-07-07T06:54:04-07:00
City Panquehue
Amount 103.76
Name Jolene
Date 2017-04-27T10:06:16-07:00
City Dijon
Amount 110.29
Name Bradley
Date 2017-11-11T07:26:54-08:00
City Neuss
Amount 94.8
Name Clementine
Date 2017-05-09T05:26:05-07:00
City Helena
Amount 96.83
Name Ava
Date 2018-04-21T00:46:05-07:00
City Kayseri
Amount 98.34
Name Basia
Date 2017-02-14T20:21:11-08:00
City Monte Santa Maria Tiberina
Amount 94.41
Name Declan
Date 2018-05-24T01:47:22-07:00
City Champlain
Amount 95.69
Name Bernard
Date 2017-07-11T09:45:20-07:00
City Blaenau Ffestiniog
Amount 100.71
Name Shelley
Date 2018-02-28T01:34:08-08:00
City Ergani
Amount 99.09
Name Amanda
Date 2017-07-28T16:09:47-07:00
City Dallas
Amount 103
Name Benedict
Date 2017-12-09T21:52:06-08:00
City Sperlinga
Amount 107.65
Name Lamar
Date 2017-11-07T09:04:12-08:00
City Stralsund
Amount 101.31
Name Aiko
Date 2016-09-06T01:58:15-07:00
City Rovereto
Amount 99.27

## Creating Data Frames

- Want to create a `DataFrame` from the XML input
- Tag names should translate to column names
- Best option is to first make a __list of dictionaries__
- One dictionary per row, then pass to `DataFrame` constructor

In [2]:
tree = ET.parse('files/sample.xml')
root = tree.getroot()
data = []
for row in root:
    # create a dictionary for each row
    datarow = {}
    for child in row:
        # dictionary key is the tag name, value is the text in the tag. This one is to create a dictionary for each row. 
        datarow[child.tag] = child.text
    data.append(datarow) 
    

#now make a dataframe from this list of dictionaries
df = pd.DataFrame(data)
print(df.head(10))
df.info()

         Name                       Date                        City  Amount
0        Hedy  2018-04-14T08:51:54-07:00                     Sijsele  105.06
1      Indigo  2018-07-07T06:54:04-07:00                   Panquehue  103.76
2      Jolene  2017-04-27T10:06:16-07:00                       Dijon  110.29
3     Bradley  2017-11-11T07:26:54-08:00                       Neuss    94.8
4  Clementine  2017-05-09T05:26:05-07:00                      Helena   96.83
5         Ava  2018-04-21T00:46:05-07:00                     Kayseri   98.34
6       Basia  2017-02-14T20:21:11-08:00  Monte Santa Maria Tiberina   94.41
7      Declan  2018-05-24T01:47:22-07:00                   Champlain   95.69
8     Bernard  2017-07-11T09:45:20-07:00          Blaenau Ffestiniog  100.71
9     Shelley  2018-02-28T01:34:08-08:00                      Ergani   99.09
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  -

## Date Fields

- In this example we have a field `Date` that is a date in ISO format
- Eg. `2017-04-27T10:06:16-07:00`
- Would be better to treat as a date rather than a string
- Python has `datetime` objects to represent dates
- Pandas provides `pd.to_datetime()` to convert a Series to datetimes (if they have the right format)

In [12]:
# read the data and coerce the Amount column to floats
df = pd.DataFrame(data).astype({'Amount': 'float64'})
# sort on date
df = df.sort_values('Date')
# create a series from the dates as datetimes
dates = pd.to_datetime(df.Date)
print(dates)
df['Date'] = dates
df.info()

43    2016-08-24 01:35:12-07:00
32    2016-08-26 20:56:02-07:00
13    2016-09-06 01:58:15-07:00
28    2016-09-10 14:10:12-07:00
86    2016-09-12 15:32:09-07:00
                ...            
74    2018-06-16 03:09:29-07:00
90    2018-06-16 22:15:26-07:00
98    2018-07-06 08:55:19-07:00
1     2018-07-07 06:54:04-07:00
21    2018-07-09 15:24:14-07:00
Name: Date, Length: 100, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 43 to 21
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Name    100 non-null    object 
 1   Date    100 non-null    object 
 2   City    100 non-null    object 
 3   Amount  100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.9+ KB


## Date field as Index

- Recall that the index of a `DataFrame` or `Series` is used to access rows
- By default this is a series of integers: 0, 1, 2, etc
- But it can be any useful identifier
- Useful to use these dates as the index
- Our data frame then becomes a [time series](http://pandas.pydata.org/pandas-docs/stable/timeseries.html)

In [16]:
df = pd.DataFrame(data).astype({'Date':'datetime64'}) # can't do this with different timezone
# sort on date
#df = df.sort_values('Date')
# make Date the index of the dataframe
#df.index = df['Date']
# remove the Date column (axis=1 means we're dropping a column, not rows)
#df.drop('Date', axis=1, inplace=True)
df2.head()
print(df2.info())

ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True

In [ ]:
# having a datetime index means we can easily select rows for a given year etc
df['2016-09']

## Getting the index of a row

- With the datetime as an index it is no longer a column in the data frame
- Available as `df.index` rather than `df['Date']`

In [ ]:
df.index[0]

## Working with rows

- Many operations can be done on a `DataFrame` as a whole
- More efficient to use these operations than iterating over rows
- But sometimes that's what you need to do
- Use the `df.iterrows()` method to iterate over rows
- Each iteration yeilds the index and the row itself

In [17]:
# make a smaller dataframe, just the first 10 rows
small = df.iloc[:10]
# this is useful if you want to print out a particular column only
for index, row in small.iterrows():
    print(index, row['Amount'])

43 97.98
32 99.04
13 99.27
28 96.53
86 107.23
70 99.06
57 100.44
65 90.59
81 103.11
45 102.78
